In [2]:
from src import Icsr
from src.utils import get_matches

import datasets
import random
from copy import deepcopy
import re

/Users/kldooste/miniconda3/envs/biodex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# load icsr extraction dataset
dataset = datasets.load_dataset("FAERS-PubMed/BioDEX-ICSR")

Using custom data configuration FAERS-PubMed--BioDEX-ICSR-a832e06da923c0a1
Found cached dataset parquet (/Users/kldooste/.cache/huggingface/datasets/FAERS-PubMed___parquet/FAERS-PubMed--BioDEX-ICSR-a832e06da923c0a1/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 3/3 [00:00<00:00, 172.95it/s]


### Manipulate the full-text
- remove front information
- remove references

In [4]:
def remove_front(text):
    if '==== Body' in text:
        text = ('\n').join(text.split('==== Body')[1:])
    return text.strip()

def remove_refs(text):
    if '==== Refs' in text:
        text = ('\n').join(text.split('==== Refs')[:-1])
    return text.strip() 

def get_fulltext_input(row):
    fulltext_filtered = remove_refs(remove_front(row['fulltext']))
    data = ['\nTITLE:', row['title'], '\nABSTRACT:', row['abstract'], '\nTEXT:', fulltext_filtered]
    return ('\n').join(data).strip()

# numbered_titles_re = r'^(\d[\.\d]*) (.*)\n'
# capitalized_titles_re = r'^([A-Z ]+)\n'

In [5]:
new_dataset = dataset.map(lambda example: {'fulltext_processed': get_fulltext_input(example)})

100%|██████████| 2407/2407 [00:00<00:00, 4550.52ex/s]


In [6]:
# # save to disk
# new_dataset.save_to_disk('../data/BioDEX-ICSR-Fulltext')
# # load from disk
# ds = datasets.load_from_disk('../data/BioDEX-ICSR-Fulltext')

In [7]:
# save to hub
new_dataset.push_to_hub("FAERS-PubMed/BioDEX-ICSR")

Pushing split train to the Hub.
Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  4.33it/s]
Pushing split test to the Hub.
Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  4.34it/s]
Pushing split validation to the Hub.
Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  2.96it/s]
